# 데이터 이해와 전처리

**이 노트북의 목표**: "내 직업이 AI에 얼마나 노출될까?"를 계산하기 위한 **데이터 준비하기**

## 필요한 데이터 2가지

1. 🤖 **MTurk AI 매핑 데이터** (mturk_mapping_matrix.csv)
   - **52개 능력**이 AI에 얼마나 영향받는지 (**Beta 값**)
   - 0에 가까우면: AI가 못함 (예: 물리적 힘)
   - 1에 가까우면: AI가 잘함 (예: 음성인식)

2. 📊 **O*NET 능력 데이터** (Abilities.xlsx)
   - **894개 직업**이 **52개 능력**을 얼마나 필요로 하는지
   - **Importance (중요도)**: 1-5점 척도
   - **Level (수준)**: 0-7점 척도
   - **Title**: 직업명 (직업 코드 변환용)


**이번 시간**: 2개 데이터를 하나로 병합하기!

---

In [991]:
import pandas as pd
import numpy as np

In [992]:
# 데이터 경로 설정
RAW_DIR = '../datas/raw'
PROCESSED_DIR = '../datas/processed'

## 1. MTurk AI 매핑 데이터 이해하기

### MTurk 데이터란?
```
MTurk(Amazon Mechanical Turk)는 아마존이 운영하는 크라우드소싱 플랫폼입니다.
연구자들은 이 플랫폼을 통해 많은 사람들에게 설문조사를 의뢰했고,
"각 AI 응용 프로그램이 52개 능력을 얼마나 대체할 수 있는가?"를 평가받았습니다.
```

In [993]:
# MTurk AI 영향도 데이터 로드
mturk = pd.read_csv(f'{RAW_DIR}/mturk_mapping_matrix.csv')
mturk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 54 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   application_id             10 non-null     int64  
 1   applications               10 non-null     object 
 2   oralcomprehension          10 non-null     float64
 3   armhandsteadiness          10 non-null     float64
 4   auditoryattention          10 non-null     float64
 5   categoryflexibility        10 non-null     float64
 6   controlprecision           10 non-null     float64
 7   deductivereasoning         10 non-null     float64
 8   depthperception            10 non-null     float64
 9   dynamicflexibility         10 non-null     float64
 10  dynamicstrength            10 non-null     float64
 11  explosivestrength          10 non-null     float64
 12  extentflexibility          10 non-null     float64
 13  farvision                  10 non-null     float64
 1

In [994]:
# 데이터 프레임 모양도 살~짝 확인하고
mturk.tail()

,application_id,applications,oralcomprehension,armhandsteadiness,auditoryattention,categoryflexibility,controlprecision,deductivereasoning,depthperception,dynamicflexibility,...,speedoflimbmovement,stamina,staticstrength,timesharing,trunkstrength,visualcolordiscrimination,visualization,wristfingerspeed,writtencomprehension,writtenexpression
5,6,reading comprehension,0.692708,0.197917,0.421875,0.817708,0.385417,0.869792,0.338542,0.197917,...,0.197917,0.197917,0.213542,0.598958,0.208333,0.442708,0.557292,0.213542,0.911458,0.833333
6,7,language modeling,0.905759,0.193717,0.821990,0.759162,0.340314,0.801047,0.282723,0.178010,...,0.167539,0.198953,0.204188,0.654450,0.172775,0.293194,0.350785,0.198953,0.827225,0.848168
7,8,translation,0.887097,0.193548,0.741935,0.688172,0.306452,0.747312,0.268817,0.172043,...,0.193548,0.209677,0.182796,0.623656,0.188172,0.338710,0.392473,0.215054,0.881720,0.854839
8,9,speech recognition,0.890110,0.230769,0.906593,0.626374,0.412088,0.604396,0.318681,0.214286,...,0.208791,0.197802,0.208791,0.664835,0.197802,0.280220,0.335165,0.214286,0.587912,0.703297
9,10,instrumental track recognition,0.604278,0.219251,0.935829,0.684492,0.368984,0.491979,0.336898,0.181818,...,0.208556,0.229947,0.208556,0.641711,0.176471,0.294118,0.310160,0.219251,0.352941,0.315508


In [995]:
# 능력은 0~1사이 값이 있는 걸 확인할 수 있다! 
mturk.describe()

,application_id,oralcomprehension,armhandsteadiness,auditoryattention,categoryflexibility,controlprecision,deductivereasoning,depthperception,dynamicflexibility,dynamicstrength,...,speedoflimbmovement,stamina,staticstrength,timesharing,trunkstrength,visualcolordiscrimination,visualization,wristfingerspeed,writtencomprehension,writtenexpression
count,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,5.50000,0.666145,0.347045,0.659127,0.766880,0.549294,0.725660,0.555437,0.289890,0.267150,...,0.316095,0.278971,0.270506,0.660934,0.242711,0.616339,0.622187,0.359952,0.642226,0.623847
std,3.02765,0.190608,0.168331,0.212247,0.084309,0.215834,0.130143,0.264407,0.115492,0.077753,...,0.147123,0.086152,0.082936,0.089714,0.067849,0.307888,0.256657,0.181728,0.180972,0.183547
min,1.00000,0.373737,0.193548,0.333333,0.626374,0.306452,0.491979,0.268817,0.172043,0.182796,...,0.167539,0.197802,0.182796,0.543590,0.172775,0.280220,0.310160,0.198953,0.352941,0.315508
25%,3.25000,0.571524,0.203250,0.482323,0.704591,0.373092,0.630847,0.323236,0.185843,0.206071,...,0.200577,0.201634,0.208615,0.605133,0.190580,0.305266,0.361207,0.214478,0.564491,0.497735
50%,5.50000,0.669431,0.287179,0.688916,0.763420,0.498352,0.742343,0.512861,0.266117,0.248535,...,0.263370,0.250871,0.227284,0.648081,0.214423,0.621354,0.680708,0.291677,0.590348,0.592657
75%,7.75000,0.839292,0.472650,0.814946,0.816564,0.705128,0.842014,0.812569,0.374204,0.309207,...,0.393376,0.342347,0.341092,0.692724,0.297786,0.926399,0.839646,0.460859,0.787086,0.800824
max,10.00000,0.905759,0.675258,0.935829,0.902564,0.917526,0.876923,0.863636,0.494845,0.396907,...,0.603093,0.427835,0.409091,0.835052,0.368687,0.939394,0.933333,0.737113,0.911458,0.854839


In [996]:
mturk['applications'].nunique()

10

### MTurk 데이터 구조

**데이터 구성:**
- 10개 AI 응용 프로그램 × 52개 능력
- 각 셀: AI가 해당 능력을 얼마나 잘 수행하는지 (0~1 점수, Beta 값)
  - 0에 가까우면: AI가 못함 (예: 신체적 힘)
  - 1에 가까우면: AI가 잘함 (예: 텍스트 이해)

**우리가 사용할 데이터:**
- 우리는 이 중 **"Language Modeling(언어 모델)" 행만** 사용합니다
- 왜냐하면 ChatGPT, Gemini, Claude 등 최근 주목받는 AI가 바로 언어 모델이기 때문입니다

In [997]:
# Language Modeling 응용 프로그램의 Beta 값 추출
lang_model_row = mturk[mturk['applications'] == 'language modeling']

# application_id, applications 빼고 나머지가 능력 컬럼
beta_values = lang_model_row.iloc[0, 2:].to_dict()
print(f"✅ Beta 값 추출 완료: {len(beta_values)}개 능력")

✅ Beta 값 추출 완료: 52개 능력


In [998]:
# Beta 값을 DataFrame으로 변환하여 정렬
beta_df = pd.DataFrame({
    '능력명': beta_values.keys(),
    'Beta값': beta_values.values()
})

beta_df_sorted = beta_df.sort_values('Beta값', ascending=False)

In [999]:
# AI가 가장 잘하는 능력 TOP 5
beta_df_sorted.head(5)

,능력명,Beta값
40,speechrecognition,0.952880
39,speechclarity,0.926702
0,oralcomprehension,0.905759
28,oralexpression,0.905759
23,memorization,0.858639


In [1000]:
# AI가 가장 어려워하는 능력 TOP 5
beta_df_sorted.tail(5)

,능력명,Beta값
46,trunkstrength,0.172775
9,explosivestrength,0.172775
42,speedoflimbmovement,0.167539
10,extentflexibility,0.167539
24,multilimbcoordination,0.162304


## 2. O*NET 2024 능력 데이터 이해하기

### O*NET이란?

**O*NET (Occupational Information Network)**
- 미국 노동부가 관리하는 **세계 최대 규모의 직업 정보 데이터베이스**
- 1,000개 가까운 직업에 대한 상세 정보 제공
- 정기적으로 업데이트 (우리는 2024년 7월 공개된 **버전 30.0** 사용)

**O*NET이 수집하는 정보:**
- 각 직업이 필요로 하는 능력(Abilities)
- 필요한 지식(Knowledge)
- 사용하는 기술(Skills)
- 업무 활동(Work Activities)
- 업무 환경(Work Context)
- ... 등 수십 가지 정보

**우리가 사용할 데이터: Abilities.xlsx**
- 각 직업이 **52개 능력**을 얼마나 필요로 하는지
- 두 가지 척도로 측정:
  - **Importance (중요도)**: 이 능력이 얼마나 중요한가?
  - **Level (수준)**: 이 능력을 얼마나 높은 수준으로 필요로 하는가?

먼저 데이터를 불러와서 전체 구조를 파악해보겠습니다.

In [ ]:
# O*NET 2024 능력 데이터 로드
onet_abil = pd.read_excel(f'{RAW_DIR}/Abilities.xlsx')

In [1002]:
onet_abil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92976 entries, 0 to 92975
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   O*NET-SOC Code      92976 non-null  object 
 1   Title               92976 non-null  object 
 2   Element ID          92976 non-null  object 
 3   Element Name        92976 non-null  object 
 4   Scale ID            92976 non-null  object 
 5   Scale Name          92976 non-null  object 
 6   Data Value          92976 non-null  float64
 7   N                   92976 non-null  int64  
 8   Standard Error      92976 non-null  float64
 9   Lower CI Bound      92976 non-null  float64
 10  Upper CI Bound      92976 non-null  float64
 11  Recommend Suppress  92976 non-null  object 
 12  Not Relevant        46488 non-null  object 
 13  Date                92976 non-null  object 
 14  Domain Source       92976 non-null  object 
dtypes: float64(4), int64(1), object(10)
memory usage: 10.

In [1003]:
onet_abil.head(2)

,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Not Relevant,Date,Domain Source
0,11-1011.00,Chief Executives,1.A.1.a.1,Oral Comprehension,IM,Importance,4.62,8,0.183,4.2664,4.9836,N,NaN,08/2023,Analyst
1,11-1011.00,Chief Executives,1.A.1.a.1,Oral Comprehension,LV,Level,4.88,8,0.125,4.6300,5.1200,N,N,08/2023,Analyst


### O*NET Abilities 데이터 구조

**핵심 구조:**
- 각 직업-능력 조합이 **2개 행**으로 저장됨
  - 1행: Importance (IM) - 중요도 점수
  - 1행: Level (LV) - 수준 점수

**우리가 사용할 컬럼:**
- `O*NET-SOC Code`: 직업 코드
- `Title`: 직업명
- `Element Name`: 능력명
- `Scale ID`: IM/LV 구분
- `Data Value`: 점수

**총 레코드 수:** 92,976개 (894 직업 × 52 능력 × 2 척도)

In [1004]:
# Importance (IM)와 Level (LV) 데이터 분리
onet_abil_im = onet_abil[onet_abil['Scale ID'] == 'IM'].copy()
onet_abil_lv = onet_abil[onet_abil['Scale ID'] == 'LV'].copy()

print(f"✅ IM: {len(onet_abil_im):,}행, LV: {len(onet_abil_lv):,}행")

✅ IM: 46,488행, LV: 46,488행


In [1005]:
# 능력명 통일 함수
def unify_ability_name(name):
    """공백, 하이픈, 언더스코어 제거 및 소문자 변환"""
    return name.lower().replace(' ', '').replace('-', '').replace('_', '')

# O*NET 능력명 통일
onet_abil_im['abil_unified'] = onet_abil_im['Element Name'].apply(unify_ability_name)
onet_abil_lv['abil_unified'] = onet_abil_lv['Element Name'].apply(unify_ability_name)

print(f"✅ 통일 완료: O*NET {onet_abil_im['abil_unified'].nunique()}개 능력")

✅ 통일 완료: O*NET 52개 능력


## 3. Importance와 Level 합치기

### 현재 문제

O*NET 데이터는 Importance와 Level이 각각 별도의 행으로 저장되어 있습니다.
아래 코드로 직접 확인해봅시다.

In [1006]:
# CEO의 'Written Expression' 능력 데이터 확인
# IM (Importance)와 LV (Level)이 별도의 행으로 저장됨
ceo_wrt = onet_abil[
    (onet_abil['O*NET-SOC Code'] == '11-1011.00') &
    (onet_abil['Element Name'] == 'Written Expression')
]
ceo_wrt[['O*NET-SOC Code', 'Element Name', 'Scale ID', 'Data Value']]

,O*NET-SOC Code,Element Name,Scale ID,Data Value
6,11-1011.00,Written Expression,IM,4.12
7,11-1011.00,Written Expression,LV,4.75


위 결과를 보면:
- **1번째 행**: Scale ID = 'IM', Data Value = 4.12 (중요도)
- **2번째 행**: Scale ID = 'LV', Data Value = 4.75(수준)

같은 직업-능력 정보가 2개 행으로 나뉘어 있습니다.

**목표:** 이 2개 행을 1개 행으로 합치기
→ 한 행에 Importance와 Level이 모두 있어야 계산하기 편함

In [1007]:
# Importance와 Level 병합
onet_merged = onet_abil_im.merge(
    onet_abil_lv[['O*NET-SOC Code', 'Element Name', 'Data Value']],
    on=['O*NET-SOC Code', 'Element Name'],
    suffixes=('_importance', '_level'),
    how='inner'
)

onet_merged.head()

,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Data Value_importance,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Not Relevant,Date,Domain Source,abil_unified,Data Value_level
0,11-1011.00,Chief Executives,1.A.1.a.1,Oral Comprehension,IM,Importance,4.62,8,0.1830,4.2664,4.9836,N,NaN,08/2023,Analyst,oralcomprehension,4.88
1,11-1011.00,Chief Executives,1.A.1.a.2,Written Comprehension,IM,Importance,4.25,8,0.1637,3.9292,4.5708,N,NaN,08/2023,Analyst,writtencomprehension,4.88
2,11-1011.00,Chief Executives,1.A.1.a.3,Oral Expression,IM,Importance,4.50,8,0.1890,4.1296,4.8704,N,NaN,08/2023,Analyst,oralexpression,4.88
3,11-1011.00,Chief Executives,1.A.1.a.4,Written Expression,IM,Importance,4.12,8,0.1250,3.8800,4.3700,N,NaN,08/2023,Analyst,writtenexpression,4.75
4,11-1011.00,Chief Executives,1.A.1.b.1,Fluency of Ideas,IM,Importance,3.88,8,0.1250,3.6300,4.1200,N,NaN,08/2023,Analyst,fluencyofideas,4.62


In [1008]:
# 컬럼명 정리
onet_merged = onet_merged.rename(columns={
    'O*NET-SOC Code': 'soc_code',
	'Title': 'title',
    'Element Name': 'ability',
    'Data Value_importance': 'importance',
    'Data Value_level': 'level'
})

# 필요한 컬럼만 선택 (ability_unified 포함)
use_cols = ['soc_code', 'title', 'ability', 'abil_unified', 'importance', 'level']
onet_merged = onet_merged[use_cols].copy()

onet_merged.head()

,soc_code,title,ability,abil_unified,importance,level
0,11-1011.00,Chief Executives,Oral Comprehension,oralcomprehension,4.62,4.88
1,11-1011.00,Chief Executives,Written Comprehension,writtencomprehension,4.25,4.88
2,11-1011.00,Chief Executives,Oral Expression,oralexpression,4.50,4.88
3,11-1011.00,Chief Executives,Written Expression,writtenexpression,4.12,4.75
4,11-1011.00,Chief Executives,Fluency of Ideas,fluencyofideas,3.88,4.62


## 4. Beta 값 추가하기

### .map() 함수란?

`.map()`은 딕셔너리를 사용해 Series의 값을 변환하는 메서드입니다.

**간단한 예제:**
```python
# 과일 가격 딕셔너리
prices = {'사과': 1000, '바나나': 500, '오렌지': 1500}

# 과일 리스트를 가격으로 변환
fruits = pd.Series(['사과', '바나나', '사과', '오렌지'])
fruit_prices = fruits.map(prices)
# 결과: [1000, 500, 1000, 1500]
```

**우리 경우:**
- `beta_values`: 능력명 → Beta 값 딕셔너리
- `onet_merged['Ability']`: 능력명 Series
- `.map(beta_values)`: 각 능력명을 Beta 값으로 변환

In [1009]:
# Beta 값 추가
# ability_unified 컬럼을 활용하여 beta_values와 매칭
onet_merged['beta'] = onet_merged['abil_unified'].map(beta_values)

## 5. 데이터 품질 확인

`.describe()`로 데이터 통계를 확인하고 결측값을 체크합니다.

In [1010]:
onet_merged.describe()

,importance,level,beta
count,46488.000000,46488.000000,46488.000000
mean,2.476618,2.208320,0.472916
std,0.913785,1.323611,0.270739
min,1.000000,0.000000,0.162304
25%,1.880000,1.250000,0.208115
50%,2.620000,2.380000,0.361257
75%,3.120000,3.120000,0.723822
max,5.000000,6.000000,0.952880


In [1011]:
onet_merged.isnull().sum()

soc_code        0
title           0
ability         0
abil_unified    0
importance      0
level           0
beta            0
dtype: int64

## 6. 최종 데이터 저장

In [1012]:
# ability_unified 컬럼 제거 (더 이상 필요 없음)
onet_merged = onet_merged.drop(columns=['abil_unified'])

onet_merged.head()

,soc_code,title,ability,importance,level,beta
0,11-1011.00,Chief Executives,Oral Comprehension,4.62,4.88,0.905759
1,11-1011.00,Chief Executives,Written Comprehension,4.25,4.88,0.827225
2,11-1011.00,Chief Executives,Oral Expression,4.50,4.88,0.905759
3,11-1011.00,Chief Executives,Written Expression,4.12,4.75,0.848168
4,11-1011.00,Chief Executives,Fluency of Ideas,3.88,4.62,0.717277


In [1013]:
# 최종 데이터 저장
output_file = f'{PROCESSED_DIR}/preprocessed_data.csv'
onet_merged.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"✅ 전처리 완료! 저장 위치: {output_file}")
print(f"📊 최종 데이터 형태: {onet_merged.shape}")

✅ 전처리 완료! 저장 위치: ../datas/processed/preprocessed_data.csv
📊 최종 데이터 형태: (46488, 6)
